In [511]:
%matplotlib inline
import os, sys

import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
import pandas as pd
import yaml

import pcse
from pcse.models import Wofost72_WLP_FD, Wofost80_NWLP_FD_beta, LINTUL3
from pcse.fileinput import CABOFileReader, YAMLCropDataProvider, PCSEFileReader
from pcse.db import NASAPowerWeatherDataProvider
from pcse.util import WOFOST72SiteDataProvider, WOFOST80SiteDataProvider
from pcse.base import ParameterProvider
from pcse.fileinput import YAMLAgroManagementReader

In [512]:
print(os.getcwd())

/home/tmrob2/PycharmProjects/farming-gym


In [513]:
data_dir = os.path.join(os.getcwd(), 'farm_gym/envs/env_data/')

In [514]:
crop_params_dir = os.path.join(data_dir, 'crop_params/')
crop = YAMLCropDataProvider(crop_params_dir)

YAMLCropDataProvider - Failed to load cache file: [Errno 2] No such file or directory: 'barley'


In [515]:
crop.get_crops_varieties()

{'barley': dict_keys(['Spring_barley_301']),
 'cassava': dict_keys(['Cassava_VanHeemst_1988']),
 'chickpea': dict_keys(['Chickpea_VanHeemst_1988']),
 'cotton': dict_keys(['Cotton_VanHeemst_1988']),
 'cowpea': dict_keys(['Cowpea_VanHeemst_1988']),
 'fababean': dict_keys(['Faba_bean_801']),
 'groundnut': dict_keys(['Groundnut_VanHeemst_1988']),
 'maize': dict_keys(['Maize_VanHeemst_1988', 'Grain_maize_201', 'Grain_maize_202', 'Grain_maize_203', 'Grain_maize_204', 'Grain_maize_205']),
 'millet': dict_keys(['Millet_VanHeemst_1988']),
 'mungbean': dict_keys(['Mungbean_VanHeemst_1988']),
 'pigeonpea': dict_keys(['Pigeonpea_VanHeemst_1988']),
 'potato': dict_keys(['Potato_701', 'Potato_702', 'Potato_703', 'Potato_704']),
 'rapeseed': dict_keys(['Oilseed_rape_1001', 'Oilseed_rape_1002', 'Oilseed_rape_1003', 'Oilseed_rape_1004']),
 'rice': dict_keys(['Rice_501', 'Rice_HYV_IR8', 'Rice_IR64616H_DS', 'Rice_IR64616H_WS', 'Rice_IR64', 'Rice_IR72', 'Rice_IR72_DS', 'Rice_IR72_WS', 'Rice_IR8A']),
 'sor

In [516]:
soil_data = os.path.join(data_dir, 'SOILD', 'EC2.NEW')
soil = CABOFileReader(soil_data)
#soil = PCSEFileReader(soil_data)

In [517]:
sited = WOFOST72SiteDataProvider(WAV=10)

In [518]:
parameterprovider = ParameterProvider(soildata=soil, cropdata=crop, sitedata=sited)

In [519]:
def get_weatherdataprovider(lat, long):
    location = (lat, long)
    return pcse.db.NASAPowerWeatherDataProvider(*location)
wdp = get_weatherdataprovider(-36.626230, 142.188370)
#wdp = get_weatherdataprovider(51.970, 5.670)
print(wdp)

Weather data provided by: NASAPowerWeatherDataProvider
--------Description---------
NASA/POWER CERES/MERRA2 Native Resolution Daily Data
----Site characteristics----
Elevation:  133.0
Latitude:  -36.626
Longitude: 142.188
Data available for 1984-01-01 - 2022-07-24
Number of missing days: 1



In [520]:
agro_dir = os.path.join(os.getcwd(), "farm_gym/envs/env_data/agro/")
agromanagement_file = os.path.join(agro_dir, 'wofost_wheat_timed_event.yaml')
agromanagement = YAMLAgroManagementReader(agromanagement_file)
print(agromanagement)

!!python/object/new:pcse.fileinput.yaml_agro_loader.YAMLAgroManagementReader
listitems:
- 2006-01-01:
    CropCalendar:
      crop_end_date: 2007-04-20
      crop_end_type: earliest
      crop_name: maize
      crop_start_date: 2006-11-01
      crop_start_type: sowing
      max_duration: 300
      variety_name: Maize_VanHeemst_1988
    StateEvents: null
    TimedEvents:
    - comment: All irrigation amounts in cm
      event_signal: irrigate
      events_table:
      - 2006-12-01:
          amount: 20
          efficiency: 0.7
      - 2006-12-15:
          amount: 20
          efficiency: 0.7
      - 2007-01-01:
          amount: 10
          efficiency: 0.7
      name: Irrigation application table
- 2007-04-30:
    CropCalendar:
      crop_end_date: 2007-12-30
      crop_end_type: earliest
      crop_name: wheat
      crop_start_date: 2007-05-01
      crop_start_type: sowing
      max_duration: 300
      variety_name: Winter_wheat_101
    StateEvents: null
    TimedEvents:
    - comme

In [521]:
wofost = Wofost72_WLP_FD(parameterprovider, wdp, agromanagement)
wofost.run_till_terminate()

In [522]:
output = wofost.get_output()
df = pd.DataFrame(output).set_index("day")
df.tail()

,DVS,LAI,TAGP,TWSO,TWLV,TWST,TWRT,TRA,RD,SM,WWLOW
day,,,,,,,,,,,
2007-12-15,1.931248,0.0,14768.472808,1519.474953,5501.923714,7747.074141,3157.308212,0.0,120.0,0.102238,12.268530
2007-12-16,1.947111,0.0,14768.472808,1519.474953,5501.923714,7747.074141,3157.308212,0.0,120.0,0.103614,12.433630
2007-12-17,1.963777,0.0,14768.472808,1519.474953,5501.923714,7747.074141,3157.308212,0.0,120.0,0.101521,12.182504
2007-12-18,1.980544,0.0,14768.472808,1519.474953,5501.923714,7747.074141,3157.308212,0.0,120.0,0.100371,12.044473
2007-12-19,2.000000,0.0,14768.472808,1519.474953,5501.923714,7747.074141,3157.308212,0.0,120.0,0.100073,12.008789


In [523]:
df.insert(0, "crop_days", output.index[:])
df.tail()


TypeError: 'builtin_function_or_method' object is not subscriptable

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(16,8))
df['LAI'].plot(ax=axes[0], title="Leaf Area Index")
df['SM'].plot(ax=axes[1], title="Root zone soil moisture")
df['TAGP'].plot(ax=axes[2], title="Total above ground production")
df['TWLV'].plot(ax=axes[3], title="Total weight leaves")
fig.autofmt_xdate()
